"[Список стран по ожидаемой продолжительности жизни](https://ru.wikipedia.org/wiki/Список_стран_по_ожидаемой_продолжительности_жизни)"<br />
"[Продолжительность жизни в субъектах Российской Федерации](https://ru.wikipedia.org/wiki/Продолжительность_жизни_в_субъектах_Российской_Федерации)"<br />
[fedstat.ru](https://www.fedstat.ru/indicator/31293) (ЕМИСС - Единая межведомственная информационно-статистическая система)

In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
pd.options.display.max_rows = 200
pd.options.display.max_columns = 50

In [3]:
# load stats about longevity per year
df = pd.read_excel('data/Rosstat_by_year_and_region.xls', skiprows=2)

print(df.shape)
df.head(2)

(108, 34)


,Unnamed: 0,Unnamed: 1,Unnamed: 2,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Российская Федерация,Оба пола,все население,69.2,68.9,67.8,65.0,63.9,64.5,65.8,66.7,67.1,65.9,65.3,65.2,65.0,64.9,65.3,65.4,66.7,67.6,68.0,68.8,68.9,69.83,70.24,70.76,70.93,71.39,71.87,72.70,72.91,73.34,71.54
1,Центральный федеральный округ,Оба пола,все население,69.5,69.2,68.3,65.6,64.2,64.9,66.5,67.4,67.6,66.4,66.1,65.8,65.6,65.7,66.3,66.5,67.6,68.4,68.8,69.7,69.9,71.19,71.43,71.93,72.10,72.72,73.07,73.89,74.01,74.54,72.57


In [4]:
# remove surrounding spaces in the column with names of regions
df['Unnamed: 0'] = df['Unnamed: 0'].map(lambda st: st.strip())

# convert region column to index
df.set_index('Unnamed: 0', inplace=True)

# for brevityh of display remove name of index column.
df.index.name = ''

# sort index to speed up further transformations
df.sort_index(inplace=True)

# patch for Ingushetia for 1990-1992
# df.loc['Республика Ингушетия', '1990':'1992'] = df.loc['Чеченская и Ингушская Республики', '1990':'1992']

print(df.shape)
df.head(2)

(108, 33)


,Unnamed: 1,Unnamed: 2,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Агинский Бурятский округ (Забайкальский край),Оба пола,все население,NaN,NaN,NaN,62.4,59.9,62.7,62.3,63.9,63.5,62.8,61.9,61.3,62.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Алтайский край,Оба пола,все население,68.9,68.4,67.7,65.0,64.0,64.3,65.3,66.0,67.8,66.8,66.6,66.3,65.6,65.7,65.5,64.6,66.5,67.1,67.4,68.4,68.4,68.97,69.11,69.77,70.01,70.44,70.74,71.1,71.11,71.61,70.19


In [5]:
def filter_regions(df_regions: 'dataframe', errors='raise'):
    """
    The function takes dataframe with regions, remove some rows and rename others.
    """
    
    ls_dropping = [
        'Центральный федеральный округ',
        'Северо-Западный федеральный округ',
        'Южный федеральный округ (по 2009 год)',
        'Южный федеральный округ (с 2010 года)',
        'Южный федеральный округ (с 29.07.2016)',
        'Северо-Кавказский федеральный округ',
        'Приволжский федеральный округ',
        'Уральский федеральный округ',
        'Сибирский федеральный округ',
        'Дальневосточный федеральный округ',
        'Крымский федеральный округ',
        'Коми-Пермяцкий округ, входящий в состав Пермского края',
        'Таймырский (Долгано-Ненецкий) автономный округ (Красноярский край)',
        'Эвенкийский автономный округ (Красноярский край)',
        'Усть-Ордынский Бурятский округ',
        'Агинский Бурятский округ (Забайкальский край)',
        'Корякский округ, входящий в состав Камчатского края',
        'Чеченская и Ингушская Республики' 
    ]

    dict_renaming = {
        'Российская Федерация': 'Россия в среднем',
        'Город Москва столица Российской Федерации город федерального значения': 'г. Москва',
        'Город Санкт-Петербург город федерального значения': 'г. Санкт-Петербург',
        'Республика Адыгея (Адыгея)': 'Республика Адыгея',
        'Город федерального значения Севастополь': 'г. Севастополь',
        'Республика Татарстан (Татарстан)': 'Республика Татарстан',
        'Чувашская Республика - Чувашия': 'Чувашская Республика (Чувашия)'
    }
    
    return df_regions.drop(ls_dropping, errors=errors) \
                     .rename(index=dict_renaming, errors=errors) \
                     .sort_index()
    
df = filter_regions(df)
print(df.shape)
df.head(2)

(88, 33)


,Unnamed: 1,Unnamed: 2,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Алтайский край,Оба пола,все население,68.9,68.4,67.7,65.0,64.0,64.3,65.3,66.0,67.8,66.8,66.6,66.3,65.6,65.7,65.5,64.6,66.5,67.1,67.4,68.4,68.4,68.97,69.11,69.77,70.01,70.44,70.74,71.10,71.11,71.61,70.19
Амурская область,Оба пола,все население,67.8,67.4,65.8,62.8,62.1,63.0,62.9,63.6,64.5,63.1,62.2,61.3,61.1,60.9,60.3,60.2,62.1,63.8,63.4,64.2,64.4,64.82,65.12,66.38,67.00,67.27,68.28,69.06,69.11,68.66,67.38


In [6]:
# select only required columns
df = df[['1991', '2000', '2014', '2019', '2020']]  # df[['2020', '2019', '2014', '2000', '1991', '1990']]

print(df.shape)
df.head(2)

(88, 5)


,1991,2000,2014,2019,2020
,,,,,
Алтайский край,68.4,66.6,70.01,71.61,70.19
Амурская область,67.4,62.2,67.00,68.66,67.38


<br />
<br />
<br />

In [7]:
# load stats about longevity per gender ('genloc' is abbreviation for 'gender and location'
genloc = pd.read_excel('data/Rosstat_by_gender_and_location_2020.xls', skiprows=2)

print(genloc.shape)
genloc.head(10)

(864, 4)


,Unnamed: 0,Unnamed: 1,Unnamed: 2,2020
0,Российская Федерация,Женщины,все население,76.43
1,Российская Федерация,Женщины,городское население,76.61
2,Российская Федерация,Женщины,сельское население,75.82
3,Российская Федерация,Мужчины,все население,66.49
4,Российская Федерация,Мужчины,городское население,66.67
5,Российская Федерация,Мужчины,сельское население,65.97
6,Российская Федерация,Оба пола,все население,71.54
7,Российская Федерация,Оба пола,городское население,71.81
8,Российская Федерация,Оба пола,сельское население,70.69
9,Центральный федеральный округ,Женщины,все население,77.24


In [8]:
# rename columns
genloc.rename(columns={'Unnamed: 0': 'region', 'Unnamed: 1': 'gender', 'Unnamed: 2': 'location'}, inplace=True)

# remove surrounding spaces in the column with names of regions
genloc['region'] = genloc['region'].map(lambda st: st.strip())

genloc.head(10)

,region,gender,location,2020
0,Российская Федерация,Женщины,все население,76.43
1,Российская Федерация,Женщины,городское население,76.61
2,Российская Федерация,Женщины,сельское население,75.82
3,Российская Федерация,Мужчины,все население,66.49
4,Российская Федерация,Мужчины,городское население,66.67
5,Российская Федерация,Мужчины,сельское население,65.97
6,Российская Федерация,Оба пола,все население,71.54
7,Российская Федерация,Оба пола,городское население,71.81
8,Российская Федерация,Оба пола,сельское население,70.69
9,Центральный федеральный округ,Женщины,все население,77.24


In [9]:
# make dataframe multi-index
genloc = genloc.sort_values(by=['region', 'gender', 'location']) \
               .set_index(['region', 'gender', 'location'])

print(genloc.shape)
genloc.head(10)

(864, 1)


2020
region           gender   location                  
Алтайский край   Женщины  все население        75.16
                          городское население  75.29
                          сельское население   74.70
                 Мужчины  все население        65.18
                          городское население  65.13
                          сельское население   64.88
                 Оба пола все население        70.19
                          городское население  70.36
                          сельское население   69.64
Амурская область Женщины  все население        72.79

In [10]:
# filter records: remove some ot them and rename others
genloc = filter_regions(genloc, errors='ignore')

print(genloc.shape)
genloc.head(10)

(792, 1)


2020
region           gender   location                  
Алтайский край   Женщины  все население        75.16
                          городское население  75.29
                          сельское население   74.70
                 Мужчины  все население        65.18
                          городское население  65.13
                          сельское население   64.88
                 Оба пола все население        70.19
                          городское население  70.36
                          сельское население   69.64
Амурская область Женщины  все население        72.79

In [11]:
# reshape dataframe, transforming the last two levels of multi-index to columns
genloc = genloc.unstack().unstack()
genloc.head(2)

2020                                               \
location         все население                  городское население           
gender                 Женщины Мужчины Оба пола             Женщины Мужчины   
region                                                                        
Алтайский край           75.16   65.18    70.19               75.29   65.13   
Амурская область         72.79   62.27    67.38               73.10   62.53   

                                                               
location                  сельское население                   
gender           Оба пола            Женщины Мужчины Оба пола  
region                                                         
Алтайский край      70.36              74.70   64.88    69.64  
Амурская область    67.83              71.89   61.68    66.36

In [12]:
# simplify heading
heading = genloc.columns
genloc.columns = map(lambda tp: '-'.join(tp[1:]), heading)

genloc.head(2)

,все население-Женщины,все население-Мужчины,все население-Оба пола,городское население-Женщины,городское население-Мужчины,городское население-Оба пола,сельское население-Женщины,сельское население-Мужчины,сельское население-Оба пола
region,,,,,,,,,
Алтайский край,75.16,65.18,70.19,75.29,65.13,70.36,74.70,64.88,69.64
Амурская область,72.79,62.27,67.38,73.10,62.53,67.83,71.89,61.68,66.36


In [13]:
# choose only required to us columns and place them in required order
genloc = genloc[[
    'все население-Оба пола',
    'все население-Мужчины',
    'все население-Женщины',
    'городское население-Оба пола',
    'сельское население-Оба пола'    
]].rename(columns={
    'все население-Оба пола': 'overall',
    'все население-Мужчины': 'males',
    'все население-Женщины': 'females',
    'городское население-Оба пола': 'urban',
    'сельское население-Оба пола': 'rural'
})


# if in some column there's 0, than change it to NaN (it's actual for Saint Petersburg)
genloc.replace(0, np.nan, inplace=True)

genloc.head(2)

,overall,males,females,urban,rural
region,,,,,
Алтайский край,70.19,65.18,75.16,70.36,69.64
Амурская область,67.38,62.27,72.79,67.83,66.36


<br />

In [14]:
# before joining two dataframes make sure correctness of transformations
assert len(df) == len(genloc), "Problem wiht length of dataframes"
assert all(df.index == genloc.index), "Problem with indexes of dataframes"
assert all(df['2020'] == genloc['overall']), "content of the column '2020' is not identical to columns 'all_population'"

In [15]:
# concatinate info from two database
df = pd.concat([genloc, df], axis='columns')
df.head(2)

,overall,males,females,urban,rural,1991,2000,2014,2019,2020
Алтайский край,70.19,65.18,75.16,70.36,69.64,68.4,66.6,70.01,71.61,70.19
Амурская область,67.38,62.27,72.79,67.83,66.36,67.4,62.2,67.00,68.66,67.38


In [16]:
# calculate difference between some columns and insert result as new columns in dataframe
df.insert(loc=3, column='females_vs_males', value=df['females']-df['males'])
df.insert(loc=6, column='urban_vs_rural', value=df['urban']-df['rural'])
df.insert(loc=8, column='1991→2000', value=df['2000']-df['1991'])
df.insert(loc=10, column='2000→2014', value=df['2014']-df['2000'])
df.insert(loc=12, column='2014→2019', value=df['2019']-df['2014'])
df.insert(loc=14, column='2019→2020', value=df['2020']-df['2019'])
df.head(2)

,overall,males,females,females_vs_males,urban,rural,urban_vs_rural,1991,1991→2000,2000,2000→2014,2014,2014→2019,2019,2019→2020,2020
Алтайский край,70.19,65.18,75.16,9.98,70.36,69.64,0.72,68.4,-1.8,66.6,3.41,70.01,1.60,71.61,-1.42,70.19
Амурская область,67.38,62.27,72.79,10.52,67.83,66.36,1.47,67.4,-5.2,62.2,4.80,67.00,1.66,68.66,-1.28,67.38


In [17]:
# sort table by 'all_population'
df.sort_values(by=['overall', '2019'], ascending=False, inplace=True)

In [18]:
# move some records to the top of dataframe (by default, column 'Россия в среднем')
# change order of some regions
indexes = df.index.to_list()
ls_change_order = ['Россия в среднем']    # 'г. Москва', 'г. Санкт-Петербург', 'г. Севастополь'
indexes = list(el for el in indexes if el not in ls_change_order)
indexes = ls_change_order + indexes
df = df.reindex(indexes)

In [19]:
ls_dropping = [
    'Архангельская область',
    'Тюменская область'
]

In [20]:
df_subjects = df.drop(index=ls_dropping)
print(df_subjects.shape)
df_subjects.fillna(' ')

(86, 16)


,overall,males,females,females_vs_males,urban,rural,urban_vs_rural,1991,1991→2000,2000,2000→2014,2014,2014→2019,2019,2019→2020,2020
Россия в среднем,71.54,66.49,76.43,9.94,71.81,70.69,1.12,68.9,-3.6,65.3,5.63,70.93,2.41,73.34,-1.80,71.54
Республика Ингушетия,81.48,77.43,84.61,7.18,81.56,80.95,0.61,,,72.0,7.42,79.42,3.98,83.40,-1.92,81.48
Республика Дагестан,76.43,73.33,79.39,6.06,77.92,75.22,2.7,72.6,-1.6,71.0,4.83,75.83,3.27,79.10,-2.67,76.43
г. Москва,76.20,72.39,79.77,7.38,76.28,71.32,4.96,69.8,0.0,69.8,6.9,76.70,1.66,78.36,-2.16,76.20
Карачаево-Черкесская Республика,74.82,70.51,78.83,8.32,74.80,74.82,-0.02,71.9,-3.5,68.4,5.51,73.91,2.30,76.21,-1.39,74.82
Кабардино-Балкарская Республика,74.37,70.16,78.27,8.11,74.80,73.92,0.88,70.5,-1.4,69.1,5.06,74.16,2.30,76.46,-2.09,74.37
Республика Северная Осетия-Алания,74.08,68.84,78.96,10.12,73.89,74.53,-0.64,70.5,-2.1,68.4,5.42,73.82,1.93,75.75,-1.67,74.08
г. Санкт-Петербург,73.99,69.30,78.10,8.80,73.99,,,69.5,-2.8,66.7,7.87,74.57,1.74,76.31,-2.32,73.99
г. Севастополь,73.58,68.70,78.27,9.57,73.59,73.09,0.5,,,,,72.28,1.25,73.53,0.05,73.58
Республика Адыгея,73.27,68.70,77.58,8.88,73.56,72.96,0.6,68.3,-0.7,67.6,4.41,72.01,1.84,73.85,-0.58,73.27


In [21]:
df_subjects.head(1)

,overall,males,females,females_vs_males,urban,rural,urban_vs_rural,1991,1991→2000,2000,2000→2014,2014,2014→2019,2019,2019→2020,2020
Россия в среднем,71.54,66.49,76.43,9.94,71.81,70.69,1.12,68.9,-3.6,65.3,5.63,70.93,2.41,73.34,-1.8,71.54


In [22]:
# create code for placing info in Wikipedia
def create_table(df, file_header, file_footer):

    def if_value(x, prec=2):
        return '-' if math.isnan(x) else f"{x:0.{prec}f}" #"{x:0.{prec}f}".format(x, prec)
    
    with open('design/' + file_header, mode='r', encoding="utf-8") as fh:
        table_header = fh.read()
    
    with open('design/' + file_footer, mode='r', encoding="utf-8") as fh:
        table_footer = fh.read()

    st = table_header
    for i in range(len(df)):
        ser = df.iloc[i]
        if ser.name == 'Россия в среднем':
            st += '\n' + '|-class=static-row-header\n' + \
                  f'|align=left|\'\'\'{ser.name}\'\'\' ' + \
                  f'|| style="text-align:center; background:#e0ffd8;"|\'\'\'{if_value(ser["overall"], 2)}\'\'\' ' + \
                  f'|| style="background:#eaf3ff;"|\'\'\'{if_value(ser["males"], 2)}\'\'\' ' + \
                  f'|| style="background:#fee7f6;"|\'\'\'{if_value(ser["females"], 2)}\'\'\' ' + \
                  f'|| style="background:#fff8dc;"|\'\'\'{if_value(ser["females_vs_males"], 2)}\'\'\' ' + \
                  f'|| style="border-left-width:2px;"|\'\'\'{if_value(ser["urban"], 2)}\'\'\' ' + \
                  f'|| \'\'\'{if_value(ser["rural"], 2)}\'\'\' ' + \
                  f'|| style="background:#fff8dc;"|\'\'\'{if_value(ser["urban_vs_rural"], 2)}\'\'\' ' + \
                  f'|| style="border-left-width:2px;"|\'\'\'{if_value(ser["1991"], 1)}\'\'\' ' + \
                  f'|| style="background:#fff8dc;"|\'\'\'{if_value(ser["1991→2000"], 1)}\'\'\' ' + \
                  f'|| \'\'\'{if_value(ser["2000"], 1)}\'\'\' ' + \
                  f'|| style="background:#fff8dc;"|\'\'\'{if_value(ser["2000→2014"], 2)}\'\'\' ' + \
                  f'|| \'\'\'{if_value(ser["2014"], 2)}\'\'\' ' + \
                  f'|| style="background:#fff8dc;"|\'\'\'{if_value(ser["2014→2019"], 2)}\'\'\' ' + \
                  f'|| \'\'\'{if_value(ser["2019"], 2)}\'\'\' ' + \
                  f'|| style="background:#fff8dc;"|\'\'\'{if_value(ser["2019→2020"], 2)}\'\'\' ' + \
                  f'|| style="text-align:center; background:#e0ffd8;"|\'\'\'{if_value(ser["2020"], 2)}\'\'\''
        else:        
            st += '\n' + '|-\n' + \
                  f'|align=left|[[{ser.name}]] ' + \
                  f'|| style="text-align:center; background:#e0ffd8;"|\'\'\'{if_value(ser["overall"], 2)}\'\'\' ' + \
                  f'|| style="background:#eaf3ff;"|{if_value(ser["males"], 2)} ' + \
                  f'|| style="background:#fee7f6;"|{if_value(ser["females"], 2)} ' + \
                  f'|| style="background:#fff8dc;"|{if_value(ser["females_vs_males"], 2)} ' + \
                  f'|| style="border-left-width:2px;"|{if_value(ser["urban"], 2)} ' + \
                  f'|| {if_value(ser["rural"], 2)} ' + \
                  f'|| style="background:#fff8dc;"|{if_value(ser["urban_vs_rural"], 2)} ' + \
                  f'|| style="border-left-width:2px;"|{if_value(ser["1991"], 1)} ' + \
                  f'|| style="background:#fff8dc;"|{if_value(ser["1991→2000"], 1)} ' + \
                  f'|| {if_value(ser["2000"], 1)} ' + \
                  f'|| style="background:#fff8dc;"|{if_value(ser["2000→2014"], 2)} ' + \
                  f'|| {if_value(ser["2014"], 2)} ' + \
                  f'|| style="background:#fff8dc;"|{if_value(ser["2014→2019"], 2)} ' + \
                  f'|| {if_value(ser["2019"], 2)} ' + \
                  f'|| style="background:#fff8dc;"|{if_value(ser["2019→2020"], 2)} ' + \
                  f'|| style="text-align:center; background:#e0ffd8;"|\'\'\'{if_value(ser["2020"], 2)}\'\'\''
    st += '\n' + table_footer
    return st

table_code = create_table(df_subjects, file_header='rosstat_header_ru.txt', file_footer='rosstat_footer_ru.txt')

In [23]:
# write the code to file
with open('output/Table code for Russian regions.txt', 'w', encoding="utf-8") as fh:
    fh.write(table_code)

# Naming of some regions are reqiered small manual asjustment for proper linking to Wikipedia pages of according redions.
# Ideally this should be make via dictionary - this code is not professional.

In [24]:
# print the code here (if it is convenient)
# print(table_code)
# print('done')

<br />
<br />
<br />

#### Auxiliary table that I decided not to use

In [25]:
ls_attention = [
    'Архангельская область',
    'Архангельская область (кроме Ненецкого автономного округа)',
    'Ненецкий автономный округ (Архангельская область)',
    'Тюменская область',
    'Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)', 
    'Ханты-Мансийский автономный округ - Югра (Тюменская область)',
    'Ямало-Ненецкий автономный округ (Тюменская область)',
]

df_auxiliary = df.loc[ls_attention]
print(df_auxiliary.shape)
df_auxiliary.fillna(' ')

(7, 16)


,overall,males,females,females_vs_males,urban,rural,urban_vs_rural,1991,1991→2000,2000,2000→2014,2014,2014→2019,2019,2019→2020,2020
Архангельская область,71.35,65.71,77.03,11.32,72.13,67.75,4.38,69.1,-6.3,62.8,7.43,70.23,2.11,72.34,-0.99,71.35
Архангельская область (кроме Ненецкого автономного округа),71.39,65.79,77.01,11.22,72.11,68.05,4.06,,,,,70.20,2.10,72.30,-0.91,71.39
Ненецкий автономный округ (Архангельская область),70.40,63.44,77.79,14.35,72.51,63.73,8.78,,,60.6,10.05,70.65,2.54,73.19,-2.79,70.40
Тюменская область,72.24,67.54,76.89,9.35,72.92,69.39,3.53,68.5,-2.6,65.9,5.6,71.50,2.54,74.04,-1.80,72.24
Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа),71.34,66.17,76.51,10.34,72.38,69.16,3.22,,,,,70.32,2.43,72.75,-1.41,71.34
Ханты-Мансийский автономный округ - Югра (Тюменская область),72.87,68.76,76.91,8.15,73.06,70.70,2.36,,,65.9,6.37,72.27,2.77,75.04,-2.17,72.87
Ямало-Ненецкий автономный округ (Тюменская область),71.91,67.54,76.21,8.67,72.59,68.73,3.86,,,66.7,5.22,71.92,2.26,74.18,-2.27,71.91


In [26]:
# create code for placing info in Wikipedia
def create_table_auxiliary(df, file_header):

    def if_value(x, prec=2):
        return '-' if math.isnan(x) else f"{x:0.{prec}f}" #"{x:0.{prec}f}".format(x, prec)
    
    with open('design/' + file_header, mode='r', encoding="utf-8") as fh:
        table_header = fh.read()

    # st = table_header
    st = table_header
    for i in range(len(df)):
        ser = df.iloc[i]       
        st += '\n' + '|-\n' + \
              f'|align=left|[[{ser.name}]] ' + \
              f'|| style="text-align:center; background:#e0ffd8;"|\'\'\'{if_value(ser["overall"], 2)}\'\'\' ' + \
              f'|| style="background:#eaf3ff;"|{if_value(ser["males"], 2)} ' + \
              f'|| style="background:#fee7f6;"|{if_value(ser["females"], 2)} ' + \
              f'|| style="background:#fff8dc;"|{if_value(ser["females_vs_males"], 2)} ' + \
              f'|| style="border-left-width:2px;"|{if_value(ser["urban"], 2)} ' + \
              f'|| {if_value(ser["rural"], 2)} ' + \
              f'|| style="background:#fff8dc;"|{if_value(ser["urban_vs_rural"], 2)} ' + \
              f'|| style="border-left-width:2px;"|{if_value(ser["1991"], 1)} ' + \
              f'|| style="background:#fff8dc;"|{if_value(ser["1991→2000"], 1)} ' + \
              f'|| {if_value(ser["2000"], 1)} ' + \
              f'|| style="background:#fff8dc;"|{if_value(ser["2000→2014"], 2)} ' + \
              f'|| {if_value(ser["2014"], 2)} ' + \
              f'|| style="background:#fff8dc;"|{if_value(ser["2014→2019"], 2)} ' + \
              f'|| {if_value(ser["2019"], 2)} ' + \
              f'|| style="background:#fff8dc;"|{if_value(ser["2019→2020"], 2)} ' + \
              f'|| style="text-align:center; background:#e0ffd8;"|\'\'\'{if_value(ser["2020"], 2)}\'\'\''
    st += '\n' + '}}'
    return st

table_code = create_table_auxiliary(df_auxiliary, file_header='rosstat_header_ru.txt')
# print(table_code)

In [27]:
# write the code to file
with open('output/Table code for Russian regions -attention.txt', 'w', encoding="utf-8") as fh:
    fh.write(table_code)

# Naming of some regions are reqiered small manual asjustment for proper linking to Wikipedia pages of according redions.
# Ideally this should be make via dictionary - this code is not professional.

<br />
<br />
<br />

#### Additional task - forming data for [ZipMapMaker](https://navizv.github.io/zimm/)

In [28]:
# create data for server of colormapcreation ZiMapMaker (https://navizv.github.io/zimm)
df_colormap = df[['2019', '2020']] \
                    .drop(index='Россия в среднем')
print(df_colormap.shape)
df_colormap.head(3)

(87, 2)


,2019,2020
Республика Ингушетия,83.40,81.48
Республика Дагестан,79.10,76.43
г. Москва,78.36,76.20


In [29]:
def filter_data_for_zipmapmaker_regions(df, errors='raise'):
    ls_dropping = [
        'Архангельская область',
        'Тюменская область'
    ]
    
    dict_renaming = {
        'Архангельская область (кроме Ненецкого автономного округа)': 'Архангельская область (кроме АО)',
        'Ненецкий автономный округ (Архангельская область)': 'Ненецкий автономный округ',        
        'Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)': 'Тюменская область (кроме двух АО)',
        'Ханты-Мансийский автономный округ - Югра (Тюменская область)': 'Ханты-Мансийский автономный округ - Югра',
        'Ямало-Ненецкий автономный округ (Тюменская область)': 'Ямало-Ненецкий автономный округ',
    }
    
    df_return = df.drop(ls_dropping, errors=errors) \
                  .rename(index=dict_renaming, errors=errors) \
                  [['2019', '2020']] \
                  .sort_values(by='2020', ascending=False)
    df_return.index.name = 'регион России'
    return df_return
    
data_for_zipmapmaker_regions = filter_data_for_zipmapmaker_regions(df_colormap)
data_for_zipmapmaker_regions

,2019,2020
регион России,,
Республика Ингушетия,83.40,81.48
Республика Дагестан,79.10,76.43
г. Москва,78.36,76.20
Карачаево-Черкесская Республика,76.21,74.82
Кабардино-Балкарская Республика,76.46,74.37
Республика Северная Осетия-Алания,75.75,74.08
г. Санкт-Петербург,76.31,73.99
г. Севастополь,73.53,73.58
Республика Адыгея,73.85,73.27


In [30]:
data_for_zipmapmaker_regions.to_csv('output/data for ZipMapMaker colormap -regions.csv', sep=';', encoding='utf-8')
print('Done')

Done


In [31]:
df.sort_values()

TypeError: sort_values() missing 1 required positional argument: 'by'

In [ ]:
def filter_regions(df_regions: 'dataframe', errors='raise'):
    """
    The function takes dataframe with regions, remove some rows and rename others.
    """
    
    ls_dropping = [
        'Центральный федеральный округ',
        'Северо-Западный федеральный округ',
        'Южный федеральный округ (по 2009 год)',
        'Южный федеральный округ (с 2010 года)',
        'Южный федеральный округ (с 29.07.2016)',
        'Северо-Кавказский федеральный округ',
        'Приволжский федеральный округ',
        'Уральский федеральный округ',
        'Сибирский федеральный округ',
        'Дальневосточный федеральный округ',
        'Крымский федеральный округ',
        'Коми-Пермяцкий округ, входящий в состав Пермского края',
        'Таймырский (Долгано-Ненецкий) автономный округ (Красноярский край)',
        'Эвенкийский автономный округ (Красноярский край)',
        'Усть-Ордынский Бурятский округ',
        'Агинский Бурятский округ (Забайкальский край)',
        'Корякский округ, входящий в состав Камчатского края',
        'Чеченская и Ингушская Республики' 
    ]

    dict_renaming = {

    }
    
    return df_regions.drop(ls_dropping, errors=errors) \
                     .rename(index=dict_renaming, errors=errors) \
                     .sort_index()
    
df = filter_regions(df)
print(df.shape)
df.head(2)

In [ ]:
df_colormap

<br />
<br />
<br />

In [ ]:
# # create csv-file with data of some years
# df_some_years = df_main_subjects[['1991', '2000', '2014', '2019', '2020']] \
#                     .drop(index='Россия в среднем')
# df_some_years.fillna(' ')

# df_some_years.to_csv('output/Longevity in Russian regions.csv')
# print('Done')